In [1]:
!pip install datasets
!pip install tqdm
!pip install pandas
!pip install rouge_score
!pip install bert_score
!pip install py7zr

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Using cached datasets-2.15.0-py3-none-any.whl.metadata (20 kB)
  Obtaining dependency information for numpy>=1.17 from https://files.pythonhosted.org/packages/20/be/46eed58d8ca60cfd0c4f3c6db3db79955f6de7d434db0f49fed2f817a6a4/numpy-1.26.2-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached numpy-1.26.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Obtaining dependency information for pyarrow>=8.0.0 from https://files.pythonhosted.org/packages/f3/0e/622d600e1dbaa0b59d9c6aa2b799902e3b0d1e853e813480092b3b5c1f6d/pyarrow-14.0.1-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cached pyarrow-14.0.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (3.0 kB)
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991

In [3]:
from datasets import load_dataset, load_metric
from tqdm import tqdm
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
results_path = '/Users/valerio/Downloads/giuseppe_vanilla.txt'
labels_path = '/Users/valerio/Downloads/giuseppe.txt'

with open(results_path, 'r') as file:
    sentences = file.read().split('\n')

predictions = list(filter(lambda x: x.strip() != '', sentences))

with open(labels_path, 'r') as file:
    sentences = file.read().split('\n')

labels = list(filter(lambda x: x.strip() != '', sentences))


In [24]:
print(len(labels))
print(len(predictions))

819
819


In [25]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(target, prediction, model_name='sentence-transformers/all-mpnet-base-v2'):
    # Load the model
    model = SentenceTransformer(model_name)

    # Encode the sentences
    embedding_target = model.encode(target, convert_to_tensor=True)
    embedding_prediction = model.encode(prediction, convert_to_tensor=True)

    # Reshape the embeddings for cosine similarity calculation
    embedding_target = embedding_target.reshape(1, -1)
    embedding_prediction = embedding_prediction.reshape(1, -1)

    # Compute cosine similarity
    similarity = cosine_similarity(embedding_target.cpu().detach().numpy(), embedding_prediction.cpu().detach().numpy())

    return similarity[0][0]



In [26]:
# '/content/drive/MyDrive/2_1_Deep_NLP/SICK_Summarization/utils/rouge.py'
metric = load_metric('../utils/rouge.py')
bertscore_metric = load_metric("bertscore", lang='en', model_type='bert-base-uncased')

results_v2=[]
for i in tqdm(range(len(predictions))):

    r = metric.compute(predictions=[predictions[i]], references=[labels[i]])
    r = {key: value.mid.fmeasure * 100 for key, value in r.items()}

    bertscore_metric.add_batch(predictions=[predictions[i]], references=[labels[i]])
    bertscore_result = bertscore_metric.compute(lang='en', model_type='bert-base-uncased')
    bertscore_result = sum(bertscore_result['f1']) / len(bertscore_result['f1'])
    
    semantic_simarity_score = calculate_similarity(predictions[i], labels[i])
    r["similarity_score"] = semantic_simarity_score
    r['bert_score'] = bertscore_result

    results_v2.append(r)

100%|██████████| 819/819 [04:19<00:00,  3.15it/s]


In [27]:
import pandas as pd



# Create a DataFrame
data = {'prediction': predictions, 'lables': labels, 'score': results_v2}
df = pd.DataFrame(data)

# Specify the Excel file path
excel_file_path = 'output_vanilla.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print(f'Data has been saved to {excel_file_path}')

Data has been saved to output_vanilla.xlsx


In [ ]:
from tqdm import tqdm
from transformers import pipeline, load_metric

# Load metrics
rouge_metric = load_metric('rouge.py')
bertscore_metric = load_metric('bertscore', lang='en', model_type='bert-base-uncased')

results = []

# Iterate through predictions and labels
for prediction, label in tqdm(zip(predictions, labels), total=len(predictions)):
    # Calculate ROUGE scores
    rouge_scores = rouge_metric.compute(predictions=[prediction], references=[label])
    rouge_scores = {key: value.mid.fmeasure * 100 for key, value in rouge_scores.items()}

    # Calculate BERTScore
    bertscore_metric.add_batch(predictions=[prediction], references=[label])
    bertscore_result = bertscore_metric.compute(lang='en', model_type='bert-base-uncased')
    bertscore_result = sum(bertscore_result['f1']) / len(bertscore_result['f1'])

    # Calculate semantic similarity
    semantic_similarity_score = calculate_similarity(prediction, label)

    # Add scores to the results dictionary
    rouge_scores['similarity_score'] = semantic_similarity_score
    rouge_scores['bert_score'] = bertscore_result

    # Append results to the list
    results.append(rouge_scores)


In [18]:
res_v2 = pd.DataFrame([predictions, labels, results_v2]).T
res_v2.columns = ['samsum_test_summary', 'candidates', 'results']
res_v2 = res_v2['results'].apply(pd.Series).merge(res_v2, left_index=True, right_index=True).drop('results', axis=1)

res_v2.to_excel('./res_v2_rouge_scores_EMOJI_M1.xlsx', encoding='utf-8')

TypeError: to_excel() got an unexpected keyword argument 'encoding'